In [ ]:
!pip install tqdm

In [ ]:
!pip install yelpapi

In [25]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
# os - for saving and loading files
# json - to work with json files
# math - to round up results
# time - to add a short pause to not overwhelm the server
import os, json, math, time

# to make yelpapi calls
from yelpapi import YelpAPI

# progress bar from tqdm_notebook
from tqdm.notebook import tqdm_notebook

In [26]:
import json
with open('/Users/X473970/.secret/yelp_api.json') as f: #change the path to match YOUR path!!
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [27]:
yelp = YelpAPI(login['api-key'], timeout_s = 5.0)

In [28]:
# set our API call parameters 
LOCATION = 'Seattle, WA 98122'
TERM = 'hamburger'

In [29]:
LOCATION.split(',')[0]

'Seattle'

In [30]:
## Specify fodler for saving data
FOLDER = 'Yelp_API/'

os.makedirs(FOLDER, exist_ok = True)
# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = FOLDER+f"{LOCATION.split(',')[0]}-{TERM}.json"

In [31]:
## from lecture notes
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  



In [33]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages



[!] Yelp_API/Seattle-hamburger.json already exists. Deleting previous file...
[i] Yelp_API/Seattle-hamburger.json not found. Saving empty list to new file.
- 0 previous results found.


24

In [35]:
# From lecture notes
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)



In [36]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,WHdXWSOkqUKmbU1ILg-6wQ,galbi-burger-broadway-caphill-seattle-2,Galbi Burger Broadway - Caphill,https://s3-media2.fl.yelpcdn.com/bphoto/V_8uP2...,False,https://www.yelp.com/biz/galbi-burger-broadway...,32,"[{'alias': 'burgers', 'title': 'Burgers'}]",5.0,"{'latitude': 47.620583, 'longitude': -122.321307}","[pickup, delivery]","{'address1': '219 Broadway E', 'address2': '',...",+12068299439,(206) 829-9439,2092.399599,NaN
1,7yyDBhwcpSvlCSjsIclnmw,feed-co-burgers-seattle-3,Feed Co. Burgers,https://s3-media4.fl.yelpcdn.com/bphoto/-YJKSy...,False,https://www.yelp.com/biz/feed-co-burgers-seatt...,253,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.0,"{'latitude': 47.6127807950165, 'longitude': -1...","[pickup, delivery]","{'address1': '1190 24th Ave', 'address2': '', ...",+12067266000,(206) 726-6000,417.797486,$$
2,NWy68_oMHOZzPsjEGW957w,ox-burger-seattle,Ox Burger,https://s3-media4.fl.yelpcdn.com/bphoto/KRu21G...,False,https://www.yelp.com/biz/ox-burger-seattle?adj...,58,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.5,"{'latitude': 47.614790479864894, 'longitude': ...","[pickup, delivery]","{'address1': '1523 E Madison St', 'address2': ...",+12064208220,(206) 420-8220,1212.144270,NaN
3,iEKfq5td_rawmJit6-a5ng,heyday-bar-and-restaurant-seattle-6,Heyday Bar & Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/MsjYwz...,False,https://www.yelp.com/biz/heyday-bar-and-restau...,272,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"{'latitude': 47.59054, 'longitude': -122.29219}",[delivery],"{'address1': '1372 31st Ave S', 'address2': ''...",+12068299816,(206) 829-9816,2521.726976,$$
4,smOVPL87Q0XHACrVMPa8qw,8oz-burger-and-co-seattle,8oz Burger & Co,https://s3-media1.fl.yelpcdn.com/bphoto/2HNKWo...,False,https://www.yelp.com/biz/8oz-burger-and-co-sea...,1778,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.0,"{'latitude': 47.61320990365378, 'longitude': -...","[pickup, delivery]","{'address1': '1401 Broadway', 'address2': None...",+12064665989,(206) 466-5989,1901.419015,$$


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
453,vuR_5poK2WVKO5xX-rnb1g,salmon-bay-café-seattle-2,Salmon Bay Café,https://s3-media4.fl.yelpcdn.com/bphoto/it09z6...,False,https://www.yelp.com/biz/salmon-bay-caf%C3%A9-...,226,"[{'alias': 'tradamerican', 'title': 'American'...",3.5,"{'latitude': 47.6642, 'longitude': -122.38292}","[pickup, delivery]","{'address1': '5109 Shilshole Ave NW', 'address...",+12067825539,(206) 782-5539,8660.144743,$$
454,PFK4kIyLfxvqnwYcrmmEBQ,brief-encounter-bellevue,Brief Encounter,https://s3-media4.fl.yelpcdn.com/bphoto/6AryCR...,False,https://www.yelp.com/biz/brief-encounter-belle...,227,"[{'alias': 'breakfast_brunch', 'title': 'Break...",3.5,"{'latitude': 47.635292, 'longitude': -122.200634}",[pickup],"{'address1': '2632 Bellevue Way NE', 'address2...",+14258228830,(425) 822-8830,7538.977561,$$
455,FZd7JmrGq8FaOS2Wb3J9iQ,the-cheesecake-factory-bellevue-bellevue,The Cheesecake Factory,https://s3-media4.fl.yelpcdn.com/bphoto/5jqghW...,False,https://www.yelp.com/biz/the-cheesecake-factor...,1086,"[{'alias': 'desserts', 'title': 'Desserts'}, {...",3.0,"{'latitude': 47.615931, 'longitude': -122.2020...",[],"{'address1': '401 Bellevue Square', 'address2'...",+14254506000,(425) 450-6000,7022.825313,$$
456,CvLFyOEAtAOmDAMxOjM1Eg,chaces-pancake-corral-bellevue,Chace's Pancake Corral,https://s3-media3.fl.yelpcdn.com/bphoto/AuNm9q...,False,https://www.yelp.com/biz/chaces-pancake-corral...,605,"[{'alias': 'tradamerican', 'title': 'American'...",3.5,"{'latitude': 47.59644, 'longitude': -122.19776}","[pickup, delivery]","{'address1': '1606 Bellevue Way SE', 'address2...",+14254548888,(425) 454-8888,7564.694626,$$
457,a84AgqGemRcwEHNR_FkcQg,puerto-vallarta-restaurant-seattle,Puerto Vallarta Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/qhQ6iU...,False,https://www.yelp.com/biz/puerto-vallarta-resta...,285,"[{'alias': 'mexican', 'title': 'Mexican'}]",3.0,"{'latitude': 47.560643999999996, 'longitude': ...","[pickup, delivery]","{'address1': '4727 California Ave SW', 'addres...",+12069377335,(206) 937-7335,8999.780621,$$
